<h1> RANSAC: Plane Fitting </h1>

In [8]:
%matplotlib notebook

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
import math
import random
from scipy import linalg
from ipywidgets import Layout





N_POINTS = 20
N_POINTS2 = 5
TARGET_X_SLOPE = 2
TARGET_y_SLOPE = 3
TARGET_OFFSET  = 5
EXTENTS = 5


TARGET_X_SLOPE2 = 12
TARGET_y_SLOPE2 = 8
TARGET_OFFSET2  = 10
EXTENTS2 = 7

NOISE =0
NOISE2 =1000



    
# create random data
xs = [np.random.uniform(2*EXTENTS)-EXTENTS for i in range(N_POINTS)]
ys = [np.random.uniform(2*EXTENTS)-EXTENTS for i in range(N_POINTS)]
zs = []
for i in range(N_POINTS):
    zs.append(xs[i]*TARGET_X_SLOPE + \
              ys[i]*TARGET_y_SLOPE + \
              TARGET_OFFSET + np.random.normal(scale=NOISE))

    
Points_without_outlier = np.zeros((3,len(xs)))
for i in range(0,len(xs)):
    Points_without_outlier [0,i] = xs[i]
    Points_without_outlier [1,i] = ys[i]
    Points_without_outlier [2,i] = zs[i]
    
    
    
# create outlier 
xs2 = [np.random.uniform(2*EXTENTS2)-EXTENTS2 for i in range(N_POINTS2)]
ys2 = [np.random.uniform(2*EXTENTS2)-EXTENTS2 for i in range(N_POINTS2)]
zs2 = []
for i in range(N_POINTS2):
    zs2.append(xs2[i]*TARGET_X_SLOPE2 + \
              ys2[i]*TARGET_y_SLOPE2 + \
              TARGET_OFFSET2 + np.random.normal(scale=NOISE))
              


Points_with_outlier = np.zeros((3,len(xs)+len(xs2)))
                  
                  
for i in range(0,len(xs)):
    Points_with_outlier[0,i] = xs[i]
    Points_with_outlier[1,i] = ys[i]
    Points_with_outlier[2,i] = zs[i]
                   
                 
for j in range(0,len(xs2)):
    Points_with_outlier[0,j+len(xs)-1] = xs2[j]
    Points_with_outlier[1,j+len(xs)-1] = ys2[j]
    Points_with_outlier[2,j+len(xs)-1] = zs2[j]
                            

    
#Points = Points_with_outlier





def getPointsToPlot(allPoints, a, b, c, d):
    #find smallest and highest x,y values of allPoints
    if c!= 0:
        largest_x = -1000
        largest_y = -1000
        smallest_x = 1000
        smallest_y = 1000
        for i in range(len(allPoints[0])):
            if allPoints[0,i] < smallest_x:
                smallest_x = allPoints[0,i]
            if allPoints[1,i] < smallest_y:
                smallest_y = allPoints[1,i]
            if allPoints[0,i] > largest_x:
                largest_x = allPoints[0,i]
            if allPoints[1,i] > largest_y:
                largest_y = allPoints[1,i]
        #calculate four points on plane
        pointsOnPlane = np.array([[smallest_x, smallest_y, 0], [smallest_x, largest_y, 0], [largest_x, smallest_y, 0], [largest_x, largest_y, 0]])
        for i in range(4):
            ax = pointsOnPlane[i,0]*a
            by = pointsOnPlane[i,1]*b
            pointsOnPlane[i,2] = (-ax- by -d)/c
        return pointsOnPlane
    if b !=0:
        largest_x = -1000
        largest_z = -1000
        smallest_x = 1000
        smallest_z = 1000
        for i in range(len(allPoints[0])):
            if allPoints[0,i] < smallest_x:
                smallest_x = allPoints[0,i]
            if allPoints[2,i] < smallest_z:
                smallest_z = allPoints[2,i]
            if allPoints[0,i] > largest_x:
                largest_x = allPoints[0,i]
            if allPoints[2,i] > largest_z:
                largest_z = allPoints[2,i]
        #calculate four points on plane
        pointsOnPlane = np.array([[smallest_x, 0, smallest_z], [smallest_x, 0, largest_z], [largest_x, 0, smallest_z], [largest_x, 0, largest_z]])
        for i in range(4):
            ax = pointsOnPlane[i,0]*a
            cz = pointsOnPlane[i,2]*c
            pointsOnPlane[i,1] = (-ax- cz -d)/b
        return pointsOnPlane
    if a !=0:
        largest_y = -1000
        largest_z = -1000
        smallest_y = 1000
        smallest_z = 1000
        for i in range(len(allPoints[0])):
            if allPoints[1,i] < smallest_y:
                smallest_y = allPoints[1,i]
            if allPoints[2,i] < smallest_z:
                smallest_z = allPoints[2,i]
            if allPoints[1,i] > largest_y:
                largest_y = allPoints[1,i]
            if allPoints[2,i] > largest_z:
                largest_z = allPoints[2,i]
        #print("corners")
        #print(largest_y)
        #print(largest_z)
        #print(smallest_y)
        #print(smallest_z)
        #calculate four points on plane
        pointsOnPlane = np.array([[ 0, smallest_y, smallest_z], [ 0, smallest_y, largest_z], [ 0, largest_y, smallest_z], [ 0, largest_y, largest_z]])
        for i in range(4):
            by = pointsOnPlane[i,1]*b
            cz = pointsOnPlane[i,2]*c
            pointsOnPlane[i,0] = (-by- cz -d)/a
        return pointsOnPlane



def getPointsOnLine(num):
    Ps = np.zeros((3,num))
    for i in range(num):
        Ps[0,i] = random.uniform(0, 20)
        Ps[1,i] = Ps[0,i] * 1.5 -2
        Ps[2,i] = Ps[0,i] * 1.5 -2
    return Ps 


 




#Points = np.array([[0,2,0,2,0,2,0,2],[2,0,0,2,2,0,0,2],[0,0,0,0,0.1,0.1,0.1,0.3]])
#print("Points: \n",Points)

#numPoints = len(Points[0])





def addNoise(Points, std):
    #verrauschen
    Ps = np.copy(Points)
    for i in range(len(Ps)):
        for j in range(len(Ps[i])):
            Ps[i,j] = Ps[i,j] + np.random.normal(0, std)
    return Ps

def outlier(Points, numOut):
    #Ausreißer einfügen
    Ps = np.copy(Points)
    for i in range(numOut):
        Ps[2,i] = Ps[2,i] + random.randint(5,15)

    return Ps

def PlaneEquaion (Points):
    v1 = p3-p1
    v2 = p2-p1
    cp = np.cross(v1,v2)
    a,b,c = cp
    d = np.dot(cp,p3)
    return a,b,c,dnumPoints
    
    
def inlier(Point,schwell, a, b, c, d):
    #get three points on plane ax + by + cz + d = 0
    distance = abs(a*Point[0] + b*Point[1]+c*Point[2]+d)/(math.sqrt(a*a+b*b+c*c))  
    #print("inlier? ", Point, distance)
    if distance  <= schwell:
        return True
    else:
        return False

    
   


def ransac(outlie,  schwell, minConsensusSize, maxIterations):

    global glob_std
    global glob_out
    global Points_without_outlier
    global Points_with_outlier
    
    Points = Points_without_outlier
    
    if outlie:
        Points = Points_with_outlier
    num = len(Points[0])
    
    maxI = maxIterations
    
    #Modell der Geraden ohne Ransac berechnen und anzeigen @without ransac
    A = np.ones((num, 4))
    for i in range(num):
        A[i,0] = Points[0,i]
        A[i,1] = Points[1,i]
        A[i,2] = Points[2,i]
    
    u,s,v = np.linalg.svd(A)
   
    x = v[3]
    
    
    
    pointsOnPlane = getPointsToPlot(Points,  x[0], x[1],x[2], x[3] )
    

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    x = np.array([[pointsOnPlane[0,0], pointsOnPlane[1,0]], [pointsOnPlane[2,0], pointsOnPlane[3,0]]])
    y = np.array([[pointsOnPlane[0,1], pointsOnPlane[1,1]], [pointsOnPlane[2,1], pointsOnPlane[3,1]]])
    z = np.array([[pointsOnPlane[0,2], pointsOnPlane[1,2]], [pointsOnPlane[2,2], pointsOnPlane[3,2]]])

    ax.plot_surface(x, y, z, rstride=20, cstride=20, alpha=0.5)
    ax.set(xlabel='x', ylabel='y', zlabel='z')

           
    ax.set_title('Modell ohne Ransac')    
    ax.scatter(Points[0,:], Points[1,:],Points[2,:], cmap='Greens');
    
    
                    
    #RANSAC anwenden
    ################
    done = False
    bestNumConsensus = 0
    bestConsensusSet = np.zeros((num,1))
    
    while not done:
        
        maxIterations = maxIterations - 1
    
        #1) Zwei zufällige Punkte auswählen   #Pick three random point 
        p1 = random.randint(0,num-1)
        p2 = p1
        p3 = p1
        second_found = False
        while not second_found:
            p2 = random.randint(0,num-1)
            if p2 != p1:
                second_found = True
        third_found = False         
        while not third_found:
            p3 = random.randint(0,num-1)
            if p3 != p1 and p3 != p2:
                third_found = True
        
        normal = np.cross((Points[:,p2]-Points[:,p1]), (Points[:,p3]-Points[:,p1]))
        x = np.array([normal[0], normal[1], normal[2], -normal.dot(Points[:,p1])])
    
    
     #3) Finde Consensus Set dieses Modells
        numConsensus = 0
        consensusSet = np.zeros((num,1))
        for i in range(num):
            if inlier(Points[:,i], schwell, x[0], x[1],x[2], x[3] ):
                consensusSet[numConsensus] = i
                numConsensus = numConsensus + 1
        
                
        if numConsensus >= minConsensusSize:
            done = True
            
        if numConsensus >= bestNumConsensus:
            bestNumConsensus =  numConsensus
            bestConsensusSet = np.copy(consensusSet)
            
        if maxIterations == 0:
            done = True
            
   
    # Ebenengleichung mit finalem Consensus Set berechnen
    A = np.ones((bestNumConsensus,4))
    for i in range(bestNumConsensus):
        #print("i: ",int(bestConsensusSet[i]))
        A[i,0] = Points[0,int(bestConsensusSet[i])]       
        A[i,1] = Points[1,int(bestConsensusSet[i])] 
        A[i,2] = Points[2,int(bestConsensusSet[i])]     
    u,s,v = np.linalg.svd(A)
   
    x = v[3]
    
    
    
    pointsOnPlane = getPointsToPlot(Points,  x[0], x[1],x[2], x[3] )
    
    
    fig2 = plt.figure()
    
    ax2 = fig2.add_subplot(111, projection='3d')

    x = np.array([[pointsOnPlane[0,0], pointsOnPlane[1,0]], [pointsOnPlane[2,0], pointsOnPlane[3,0]]])
    y = np.array([[pointsOnPlane[0,1], pointsOnPlane[1,1]], [pointsOnPlane[2,1], pointsOnPlane[3,1]]])
    z = np.array([[pointsOnPlane[0,2], pointsOnPlane[1,2]], [pointsOnPlane[2,2], pointsOnPlane[3,2]]])

    ax2.plot_surface(x, y, z, rstride=20, cstride=20, alpha=0.5)
    ax2.set(xlabel='x', ylabel='y', zlabel='z')
    
                    
   
    #Modell der Geraden mit Ransac anzeigen        
    ax2.set_title('Modell mit Ransac')  
    ax2.scatter(Points[0,:], Points[1,:],Points[2,:], cmap='Greens');
    
    
    
    
    plt.show()
    
    #print("Anzahl der benötigten Iterationen:")
    #print(maxI - maxIterations)
    
    
style = {'description_width': 'initial'}
interact(ransac, outlie=widgets.Checkbox(value=True, description="Outliers",disable=False,indent=False),   schwell=widgets.FloatSlider(min=0, max=5, step=0.5, value=0.5, description='Schwellwert (Abstand zum Modell)', style=style, layout=Layout(width='100%')), minConsensusSize=widgets.IntSlider(min=0, max=20, step=1, value=5, description='Mindestgröße Consensus Set', style=style, layout=Layout(width='100%')), maxIterations=widgets.IntSlider(min=1, max=380, step=1, value=10, description='Maximale Anzahl an Iterationen', style=style, layout=Layout(width='100%')));
  

interactive(children=(Checkbox(value=True, description='Outliers', indent=False), FloatSlider(value=0.5, descr…